<a href="https://colab.research.google.com/github/mpppk/colaboratory/blob/master/sutaba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import tensorflow as tf
# tf.test.gpu_device_name()
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# !apt-get update && apt-get install imagemagick
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'

0.02971days (2566.94sec)


In [2]:
import os
from glob import glob
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np
import time

# 分類するクラス
classes = ["sutaba","ramen", "other"]
nb_classes = len(classes)
#画像の大きさを設定
img_width, img_height = 224, 224

# トレーニング用とバリデーション用の画像格納先（パスは自分で設定してください）
base_path = "/content/gdrive/My Drive/ColabNotebooks/sutaba"
train_data_dir = base_path + '/train'
test_data_dir = base_path + '/test'
aug_train_data_dir = base_path + '/aug_train'
model_dir = base_path + '/models'
# os.mkdir(aug_train_data_dir)

#トレーニングデータ用の画像数
def find_all_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            yield os.path.join(root, file)

def count_file_num(p: str) -> int:
  cnt = 0
  for f in find_all_files(p):
    cnt += 1
  return cnt

#バッチサイズ
batch_size = 300
#エポック数
nb_epoch = 100
validation_rate = 0.1

sutaba_train_num = count_file_num(os.path.join(train_data_dir, classes[0]))
ramen_train_num = count_file_num(os.path.join(train_data_dir, classes[1]))
other_train_num = count_file_num(os.path.join(train_data_dir, classes[2]))
class_weight = {0: other_train_num/sutaba_train_num, 1: other_train_num/ramen_train_num, 2: 1}
nb_train_samples = sutaba_train_num + ramen_train_num + other_train_num

sutaba_test_num = count_file_num(os.path.join(test_data_dir, classes[0]))
ramen_test_num = count_file_num(os.path.join(test_data_dir, classes[1]))
other_test_num = count_file_num(os.path.join(test_data_dir, classes[2]))
nb_test_samples = sutaba_test_num + ramen_test_num + other_test_num

print(sutaba_train_num, ramen_train_num, other_train_num, class_weight)
print(sutaba_test_num, ramen_test_num, other_test_num)

def get_latest_modified_file_path(dirname):
  target = os.path.join(dirname, '*')
  files = [(f, os.path.getmtime(f)) for f in glob(target)]
  latest_modified_file_path = sorted(files, key=lambda files: files[1])[-1]
  return latest_modified_file_path[0]

def create_vgg16_from_weights(weights_path: str):
  vgg16_model = create_vgg16()
  vgg16_model.load_weights(get_latest_modified_file_path(weights_path))
  return vgg16_model


def create_vgg16():
  # VGG16のロード。FC層は不要なので include_top=False
  input_tensor = Input(shape=(img_width, img_height, 3))
  vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

  # VGG16の図の緑色の部分（FC層）の作成
  top_model = Sequential()
  top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
  top_model.add(Dense(256, activation='relu'))
  top_model.add(Dropout(0.5))
  top_model.add(Dense(nb_classes, activation='softmax'))

  # VGG16とFC層を結合してモデルを作成（完成図が上の図）
  # vgg_model = Model()
  vgg_model = Model(input=vgg16.input, output=top_model(vgg16.output))

  # VGG16の図の青色の部分は重みを固定（frozen）
  for layer in vgg_model.layers[:15]:
      layer.trainable = False

  # 多クラス分類を指定
  vgg_model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
            metrics=['accuracy'])
  return vgg_model

train_datagen = ImageDataGenerator(
  rescale=1.0 / 255,
  #すでに画像の水増し済みの方は、下記２行は必要ありません。
  zoom_range=0.2,
  horizontal_flip=True,
  validation_split=validation_rate
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
  train_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=batch_size,
  # save_to_dir=aug_train_data_dir,
  shuffle=True,
  subset='training'
)
validation_generator = train_datagen.flow_from_directory(
  train_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=batch_size,
  # save_to_dir=aug_train_data_dir,
  shuffle=True,
  subset='validation'
)

Using TensorFlow backend.


900 550 2387 {0: 2.652222222222222, 1: 4.34, 2: 1}
73 67 70
Found 3454 images belonging to 3 classes.
Found 383 images belonging to 3 classes.


In [0]:
!mogrify \
  -path /content/gdrive/'My Drive'/ColabNotebooks/sutaba/train/sutaba \
  -define jpeg:size=224x224 \
  -thumbnail 224x224^ \
  -gravity center \
  -extent 224x224 \
  /content/gdrive/'My Drive'/ColabNotebooks/sutaba/train/sutaba-original/*.jpg
# convert -define jpeg:size=200x200 original.jpeg  -thumbnail 100x100^ -gravity center -extent 100x100  thumbnail.jpeg

In [0]:

import keras
vgg_model = create_vgg16_from_weights(model_dir)
n = datetime.datetime.now()
nstr = f'{n.year}-{n.month:02}-{n.day:02}_{n.hour:02}-{n.minute:02}-{n.second:02}'
fpath = base_path + f'/models/{nstr}' + 'weights.{epoch:02d}-{loss:.2f}-{acc:.2f}-{val_loss:.2f}-{val_acc:.2f}.hdf5'
cp_cb = keras.callbacks.ModelCheckpoint(filepath=fpath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
# Fine-tuning
history = vgg_model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    # samples_per_epoch=nb_train_samples,
    samples_per_epoch=len(train_generator.classes),
    nb_epoch=nb_epoch,
    callbacks=[cp_cb],
    nb_val_samples=len(validation_generator.classes),
    class_weight=class_weight
    )

W0810 03:26:11.347245 140710279047040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0810 03:26:11.366352 140710279047040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 03:26:11.384002 140710279047040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 03:26:11.431067 140710279047040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58892288/58889256 [==============================] - 1s 0us/step


W0810 03:26:12.485852 140710279047040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0810 03:26:12.487463 140710279047040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0810 03:26:14.754296 140710279047040 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: UserWarning: Update your `Model` call to the Keras 2 API: `Model(i

NameError: ignored

In [3]:
test_generator = test_datagen.flow_from_directory(
  test_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=1,
  shuffle=False)

import pandas as pd
vgg_model = create_vgg16_from_weights(model_dir)
loss = vgg_model.predict_generator(test_generator, steps=len(test_generator.classes), verbose=1)
prob = pd.DataFrame(loss, columns=classes)
prob['predict'] = prob.idxmax(axis=1)
prob['actual'] = [classes[c] for c in test_generator.classes]
prob['path'] = test_generator.filenames
prob.to_csv('results.csv')

W0820 06:20:25.875839 139936511432576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0820 06:20:25.912021 139936511432576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0820 06:20:25.920267 139936511432576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0820 06:20:25.958105 139936511432576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Found 210 images belonging to 3 classes.
58892288/58889256 [==============================] - 2s 0us/step


W0820 06:20:28.380908 139936511432576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0820 06:20:28.381812 139936511432576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0820 06:20:31.646105 139936511432576 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: UserWarning: Update your `Model` call to the Keras 2 API: `Model(i

210/210 [==============================] - 69s 329ms/step
True     186
False     24
dtype: int64


In [22]:
result = (prob['predict'] == prob['actual']).value_counts()
print("all:", result[True] / len(prob) * 100, "%")

sutaba_prob = prob[prob['actual'] == 'sutaba']
sutaba_result = (sutaba_prob['predict'] == sutaba_prob['actual']).value_counts()
print("sutaba:", sutaba_result[True] / len(sutaba_prob) * 100, "%")

ramen_prob = prob[prob['actual'] == 'ramen']
ramen_result = (ramen_prob['predict'] == ramen_prob['actual']).value_counts()
print("ramen:", ramen_result[True] / len(ramen_prob) * 100, "%")

all: 88.57142857142857 %
sutaba: 90.41095890410958 %
ramen: 83.5820895522388 %
